In [1]:
import pandas as pd
from datetime import datetime, date, timedelta

In [2]:
adls = pd.read_csv("outreachAppointmentAdls.csv")
crfs = pd.read_csv("ClientReviewFormRemovingData.csv")
adls['Date'] = pd.to_datetime(adls['VisitDate']).dt.date
crfs['Date'] = pd.to_datetime(crfs['Date of Review']).dt.date

In [3]:
def breakDown(x,y,monthsBefore, repeatData, noCrfsIncluded, cushion):
    dayLen = monthsBefore * 30
    lastDate = max(x['Date'])
    xs = []
    ys = []
    for i, row in x.iterrows():
        endFrame = row['Date'] + timedelta(days= dayLen)
        if endFrame > lastDate:
            break
        currDf = x[(x['Date'] >= row['Date']) & (x['Date'] <= endFrame)]
        hasHospitilizations = len(y[(y['Date'] <= endFrame + timedelta(days = 30 + cushion)) & (y['Date'] >= endFrame + timedelta(days = 30 - cushion)) & (y['Any Hospitalizations in the last 30 days'] == 'Yes')]) > 0
        xs.append(currDf)
        ys.append(int(hasHospitilizations))
    return xs,ys

In [4]:
## Code to create datasets
## options
monthsBefore = 2 ## number of months used for prediction
repeatData = True ## if patient has a positive in march we use data from feb and march to correspond to
noCrfsIncluded = False ## if the patient has no adls do we treat it as no hospitilizations or omit them
cushion = 7 ## number of days for the end of the frame to result in hospitilization


crfsDf = crfs
adlsDf = adls
crfsID = 'Masked Client ID'
adlsID = 'DeIdentify ID'
crfsDate = 'Date'
adlsDate = 'Date'


IDDfs = {}
xDfs = []
yVals = []
numpys ={}
count = 0
for clientId in adlsDf[adlsID].unique():
    tempx = adlsDf[adlsDf[adlsID] == clientId].sort_values(by = adlsDate)
    tempy = crfsDf[crfsDf[crfsID] == clientId].sort_values(by = crfsDate)
    IDDfs[clientId] = (tempx,tempy)
    x,y = breakDown(tempx,tempy,monthsBefore, repeatData, noCrfsIncluded, cushion)
    xDfs += x
    yVals += y
    count += 1
    print(count / len(adlsDf[adlsID].unique()))

0.006024096385542169
0.012048192771084338
0.018072289156626505
0.024096385542168676
0.030120481927710843
0.03614457831325301
0.04216867469879518
0.04819277108433735
0.05421686746987952
0.060240963855421686
0.06626506024096386
0.07228915662650602
0.0783132530120482
0.08433734939759036
0.09036144578313253
0.0963855421686747
0.10240963855421686
0.10843373493975904
0.1144578313253012
0.12048192771084337
0.12650602409638553
0.13253012048192772
0.13855421686746988
0.14457831325301204
0.15060240963855423
0.1566265060240964
0.16265060240963855
0.1686746987951807
0.1746987951807229
0.18072289156626506
0.18674698795180722
0.1927710843373494
0.19879518072289157
0.20481927710843373
0.21084337349397592
0.21686746987951808
0.22289156626506024
0.2289156626506024
0.23493975903614459
0.24096385542168675
0.2469879518072289
0.25301204819277107
0.25903614457831325
0.26506024096385544
0.2710843373493976
0.27710843373493976
0.28313253012048195
0.2891566265060241
0.29518072289156627
0.30120481927710846
0.307

In [5]:
print(len(xDfs))

46764


In [14]:
freqDf = pd.DataFrame()
for i in range(len(xDfs)):
    row = xDfs[i].iloc[:,5:78].sum(axis =  0)
    
    ## get the quarters -- you can comment out if you don't want
    startDate = min(xDfs[i]['Date'])
    endDate = max(xDfs[i]['Date'])
    q1end = startDate + timedelta(days = 15)
    q2end = q1end + timedelta(days = 15)
    q3end = q2end + timedelta(days = 15)
    
    row = row.append(xDfs[i][(xDfs[i]['Date'] >= startDate) &(xDfs[i]['Date'] <= q1end)].iloc[:,5:78].sum(axis =  0), ignore_index=True)
    row = row.append(xDfs[i][(xDfs[i]['Date'] >= q1end) &(xDfs[i]['Date'] <= q2end)].iloc[:,5:78].sum(axis =  0), ignore_index=True)
    row = row.append(xDfs[i][(xDfs[i]['Date'] >= q2end) &(xDfs[i]['Date'] <= q3end)].iloc[:,5:78].sum(axis =  0), ignore_index=True)
    row = row.append(xDfs[i][(xDfs[i]['Date'] >= q3end) &(xDfs[i]['Date'] <= endDate)].iloc[:,5:78].sum(axis =  0), ignore_index=True)
    
     ## get the standard deviations --- you cant comment out if you don't want
    stds =[]
    for col in xDfs[i].iloc[:,5:78]:
        stds.append(xDfs[i].iloc[:,5:78][col].std())
    row.append(pd.Series(stds))
    
    row = row.append(pd.Series(yVals[i], index = ['hasHospitilization']))
    
    freqDf = freqDf.append(row, ignore_index = True)
    
    print(i/len(xDfs))

0.0
2.1383970575656488e-05
4.2767941151312976e-05
6.415191172696946e-05
8.553588230262595e-05
0.00010691985287828244
0.00012830382345393893
0.0001496877940295954
0.0001710717646052519
0.00019245573518090838
0.00021383970575656488
0.00023522367633222136
0.00025660764690787786
0.00027799161748353436
0.0002993755880591908
0.0003207595586348473
0.0003421435292105038
0.0003635274997861603
0.00038491147036181676
0.00040629544093747326
0.00042767941151312976
0.00044906338208878626
0.0004704473526644427
0.0004918313232400993
0.0005132152938157557
0.0005345992643914122
0.0005559832349670687
0.0005773672055427252
0.0005987511761183816
0.0006201351466940382
0.0006415191172696946
0.0006629030878453512
0.0006842870584210076
0.0007056710289966641
0.0007270549995723206
0.0007484389701479771
0.0007698229407236335
0.0007912069112992901
0.0008125908818749465
0.0008339748524506031
0.0008553588230262595
0.000876742793601916
0.0008981267641775725
0.000919510734753229
0.0009408947053288854
0.000962278675904

0.008232828671627748
0.008254212642203404
0.008275596612779062
0.008296980583354718
0.008318364553930374
0.00833974852450603
0.008361132495081686
0.008382516465657344
0.008403900436233
0.008425284406808656
0.008446668377384312
0.008468052347959968
0.008489436318535626
0.008510820289111282
0.008532204259686938
0.008553588230262595
0.008574972200838252
0.008596356171413909
0.008617740141989565
0.00863912411256522
0.008660508083140877
0.008681892053716535
0.008703276024292191
0.008724659994867847
0.008746043965443503
0.00876742793601916
0.008788811906594817
0.008810195877170473
0.00883157984774613
0.008852963818321785
0.008874347788897443
0.0088957317594731
0.008917115730048756
0.008938499700624412
0.008959883671200068
0.008981267641775726
0.009002651612351382
0.009024035582927038
0.009045419553502694
0.00906680352407835
0.009088187494654008
0.009109571465229664
0.00913095543580532
0.009152339406380976
0.009173723376956633
0.00919510734753229
0.009216491318107947
0.009237875288683603
0.00

0.01667949704901206
0.016700881019587718
0.016722264990163372
0.01674364896073903
0.016765032931314688
0.016786416901890342
0.016807800872466
0.016829184843041654
0.016850568813617312
0.01687195278419297
0.016893336754768624
0.016914720725344282
0.016936104695919937
0.016957488666495595
0.016978872637071252
0.017000256607646907
0.017021640578222565
0.017043024548798223
0.017064408519373877
0.017085792489949535
0.01710717646052519
0.017128560431100847
0.017149944401676505
0.01717132837225216
0.017192712342827817
0.01721409631340347
0.01723548028397913
0.017256864254554787
0.01727824822513044
0.0172996321957061
0.017321016166281754
0.01734240013685741
0.01736378410743307
0.017385168078008724
0.017406552048584382
0.017427936019160036
0.017449319989735694
0.017470703960311352
0.017492087930887006
0.017513471901462664
0.01753485587203832
0.017556239842613976
0.017577623813189634
0.01759900778376529
0.017620391754340946
0.0176417757249166
0.01766315969549226
0.017684543666067917
0.0177059276

0.025211701308699
0.025233085279274654
0.025254469249850312
0.02527585322042597
0.025297237191001624
0.025318621161577282
0.025340005132152937
0.025361389102728595
0.025382773073304252
0.025404157043879907
0.025425541014455565
0.02544692498503122
0.025468308955606877
0.025489692926182535
0.02551107689675819
0.025532460867333847
0.0255538448379095
0.02557522880848516
0.025596612779060817
0.02561799674963647
0.02563938072021213
0.025660764690787787
0.02568214866136344
0.0257035326319391
0.025724916602514754
0.02574630057309041
0.02576768454366607
0.025789068514241724
0.025810452484817382
0.025831836455393036
0.025853220425968694
0.025874604396544352
0.025895988367120006
0.025917372337695664
0.02593875630827132
0.025960140278846976
0.025981524249422634
0.02600290821999829
0.026024292190573946
0.0260456761611496
0.02606706013172526
0.026088444102300917
0.02610982807287657
0.02613121204345223
0.026152596014027883
0.02617397998460354
0.0261953639551792
0.026216747925754853
0.0262381318963305

0.03380805748011291
0.033829441450688565
0.03385082542126422
0.03387220939183987
0.033893593362415535
0.03391497733299119
0.033936361303566843
0.033957745274142505
0.03397912924471816
0.034000513215293814
0.034021897185869475
0.03404328115644513
0.034064665127020784
0.034086049097596445
0.0341074330681721
0.034128817038747754
0.03415020100932341
0.03417158497989907
0.034192968950474724
0.03421435292105038
0.03423573689162604
0.034257120862201694
0.03427850483277735
0.03429988880335301
0.034321272773928664
0.03434265674450432
0.03436404071507997
0.034385424685655634
0.03440680865623129
0.03442819262680694
0.034449576597382604
0.03447096056795826
0.03449234453853391
0.034513728509109574
0.03453511247968523
0.03455649645026088
0.03457788042083654
0.0345992643914122
0.03462064836198785
0.03464203233256351
0.03466341630313917
0.03468480027371482
0.03470618424429048
0.03472756821486614
0.034748952185441793
0.03477033615601745
0.03479172012659311
0.034813104097168764
0.03483448806774442
0.034

...

0.9565691557608417
0.9565905397314173
0.956611923701993
0.9566333076725686
0.9566546916431443
0.9566760756137199
0.9566974595842956
0.9567188435548712
0.956740227525447
0.9567616114960226
0.9567829954665983
0.9568043794371739
0.9568257634077495
0.9568471473783252
0.9568685313489008
0.9568899153194765
0.9569112992900521
0.9569326832606279
0.9569540672312035
0.9569754512017792
0.9569968351723548
0.9570182191429305
0.9570396031135061
0.9570609870840818
0.9570823710546574
0.957103755025233
0.9571251389958088
0.9571465229663844
0.9571679069369601
0.9571892909075357
0.9572106748781114
0.957232058848687
0.9572534428192627
0.9572748267898383
0.957296210760414
0.9573175947309897
0.9573389787015653
0.957360362672141
0.9573817466427166
0.9574031306132923
0.9574245145838679
0.9574458985544436
0.9574672825250192
0.9574886664955949
0.9575100504661705
0.9575314344367463
0.9575528184073219
0.9575742023778975
0.9575955863484732
0.9576169703190488
0.9576383542896245
0.9576597382602001
0.9576811222307758

0.9658497989906766
0.9658711829612523
0.9658925669318279
0.9659139509024035
0.9659353348729792
0.9659567188435548
0.9659781028141305
0.9659994867847062
0.9660208707552819
0.9660422547258575
0.9660636386964332
0.9660850226670088
0.9661064066375845
0.9661277906081601
0.9661491745787357
0.9661705585493114
0.966191942519887
0.9662133264904628
0.9662347104610384
0.9662560944316141
0.9662774784021897
0.9662988623727654
0.966320246343341
0.9663416303139167
0.9663630142844923
0.9663843982550681
0.9664057822256437
0.9664271661962193
0.966448550166795
0.9664699341373706
0.9664913181079463
0.9665127020785219
0.9665340860490976
0.9665554700196732
0.966576853990249
0.9665982379608246
0.9666196219314003
0.9666410059019759
0.9666623898725515
0.9666837738431272
0.9667051578137028
0.9667265417842785
0.9667479257548541
0.9667693097254298
0.9667906936960055
0.9668120776665812
0.9668334616371568
0.9668548456077325
0.9668762295783081
0.9668976135488837
0.9669189975194594
0.966940381490035
0.966961765460610

0.9751304422205115
0.9751518261910872
0.9751732101616628
0.9751945941322385
0.9752159781028141
0.9752373620733897
0.9752587460439655
0.9752801300145411
0.9753015139851168
0.9753228979556924
0.9753442819262681
0.9753656658968437
0.9753870498674194
0.975408433837995
0.9754298178085707
0.9754512017791463
0.975472585749722
0.9754939697202977
0.9755153536908733
0.975536737661449
0.9755581216320246
0.9755795056026003
0.9756008895731759
0.9756222735437516
0.9756436575143272
0.975665041484903
0.9756864254554786
0.9757078094260543
0.9757291933966299
0.9757505773672055
0.9757719613377812
0.9757933453083568
0.9758147292789325
0.9758361132495081
0.9758574972200839
0.9758788811906595
0.9759002651612352
0.9759216491318108
0.9759430331023865
0.9759644170729621
0.9759858010435377
0.9760071850141134
0.976028568984689
0.9760499529552648
0.9760713369258404
0.9760927208964161
0.9761141048669917
0.9761354888375674
0.976156872808143
0.9761782567787187
0.9761996407492943
0.9762210247198699
0.9762424086904457

0.9844110854503464
0.9844324694209221
0.9844538533914977
0.9844752373620734
0.984496621332649
0.9845180053032248
0.9845393892738004
0.984560773244376
0.9845821572149517
0.9846035411855273
0.984624925156103
0.9846463091266786
0.9846676930972543
0.9846890770678299
0.9847104610384056
0.9847318450089813
0.984753228979557
0.9847746129501326
0.9847959969207082
0.9848173808912839
0.9848387648618595
0.9848601488324352
0.9848815328030108
0.9849029167735865
0.9849243007441622
0.9849456847147379
0.9849670686853135
0.9849884526558892
0.9850098366264648
0.9850312205970405
0.9850526045676161
0.9850739885381917
0.9850953725087674
0.985116756479343
0.9851381404499188
0.9851595244204944
0.9851809083910701
0.9852022923616457
0.9852236763322214
0.985245060302797
0.9852664442733727
0.9852878282439483
0.9853092122145239
0.9853305961850997
0.9853519801556753
0.985373364126251
0.9853947480968266
0.9854161320674023
0.9854375160379779
0.9854589000085536
0.9854802839791292
0.985501667949705
0.9855230519202806
0

0.993713112650757
0.9937344966213326
0.9937558805919083
0.9937772645624839
0.9937986485330597
0.9938200325036353
0.993841416474211
0.9938628004447866
0.9938841844153622
0.9939055683859379
0.9939269523565135
0.9939483363270892
0.9939697202976648
0.9939911042682406
0.9940124882388162
0.9940338722093919
0.9940552561799675
0.9940766401505432
0.9940980241211188
0.9941194080916944
0.9941407920622701
0.9941621760328457
0.9941835600034215
0.9942049439739971
0.9942263279445728
0.9942477119151484
0.9942690958857241
0.9942904798562997
0.9943118638268754
0.994333247797451
0.9943546317680267
0.9943760157386023
0.994397399709178
0.9944187836797537
0.9944401676503293
0.994461551620905
0.9944829355914806
0.9945043195620563
0.9945257035326319
0.9945470875032076
0.9945684714737832
0.994589855444359
0.9946112394149346
0.9946326233855102
0.9946540073560859
0.9946753913266615
0.9946967752972372
0.9947181592678128
0.9947395432383885
0.9947609272089641
0.9947823111795399
0.9948036951501155
0.9948250791206912

In [15]:
print(freqDf.shape)
freqDf.to_csv("frequencies.csv", index = True)

(46764, 366)
